# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
# for sql lite db
from sqlalchemy import create_engine
# for pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
# NLP libraries
import re
import nltk
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
# library for multi target classification
from sklearn.multioutput import MultiOutputClassifier
# libraries for estimators
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# libraries for NLP feature extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# libraries for split in test and training data
from sklearn.model_selection import train_test_split, GridSearchCV
# libraries for model evaluation
from sklearn.metrics import classification_report
# pickle library to save machine learning model
import pickle

# keep workspace awake using a module from the Image Classifier Project
from workspace_utils import active_session

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)

In [3]:
# Definition of text and categories
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [4]:
X = df["message"].values
Y = df.drop(["id", "message", "original", "genre"], axis=1).values

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    words = word_tokenize(text)
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for word in words:
        lemmatized_word = lemmatizer.lemmatize(word)
        clean_tokens.append(lemmatized_word)
    return clean_tokens

In [6]:
# for testing only
test_string = df["message"].values[0]
print(test_string)
clean_tokens = tokenize(test_string)
clean_tokens

Weather update - a cold front from Cuba that could pass over Haiti


['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
#forest = RandomForestClassifier(n_estimators=100, random_state=1)
#classifier = MultiOutputClassifier(forest, n_jobs=-1)
# classifier choice: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
# recommendation is either Linear SVC or Naive Bayes

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    #('clf', MultiOutputClassifier(LinearSVC(), n_jobs=-1))
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [10]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
# predict on test data
y_pred = pipeline.predict(X_test)

output_categories = df.drop(["id", "message", "original", "genre"], axis=1).columns

# Todo:  iterating through the category columns
index = 0
for category in output_categories:
    print("output category in column {}: {}".format(index, category))
    evaluation_report = classification_report(y_test[:,index], y_pred[:,index])
    index += 1
    print(evaluation_report)

output category in column 0: related
             precision    recall  f1-score   support

          0       0.63      0.47      0.54      1529
          1       0.85      0.91      0.88      4975
          2       0.18      0.24      0.21        50

avg / total       0.79      0.80      0.80      6554

output category in column 1: request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      5459
          1       0.78      0.43      0.55      1095

avg / total       0.88      0.88      0.87      6554

output category in column 2: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6524
          1       0.00      0.00      0.00        30

avg / total       0.99      1.00      0.99      6554

output category in column 3: aid_related
             precision    recall  f1-score   support

          0       0.73      0.88      0.80      3808
          1       0.77      0.55      0.64   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {#'clf__estimator': [RandomForestClassifier()],
     #'clf__estimator__n_estimators': [50, 100, 200],
     #'clf__estimator__criterion': ["gini", "entropy"]     
     #'clf__estimator__bootstrap': [True, False],
     #'clf__estimator__max_depth': [10, 50, 100, None],
     #'clf__estimator__max_features': ['auto', 'sqrt'],
     'clf__estimator__min_samples_leaf': [1, 2, 4],
     'clf__estimator__min_samples_split': [2, 5, 10],
     #'clf__estimator__n_jobs':[-1]
    }

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
# converting to numpy array due to error from GridSearch
X_train_debug = np.asarray(X_train)
y_train_debug = np.asarray(y_train)

In [ ]:
# keep workspace awake using a module from the Image Classifier Project
from workspace_utils import active_session

with active_session():
    # do long-running work here
    cv.fit(X_train_debug, y_train_debug)

print("\nBest Parameters:", cv.best_params_)

cv.best_score_

best_model = cv.best_estimator_

y_pred_gridsearch = best_model.predict(X_test)

# Todo:  iterating through the category columns
index = 0
for category in output_categories:
    print("output category in column {}: {}".format(index, category))
    evaluation_report = classification_report(y_test[:,index], y_pred_gridsearch[:,index])
    index += 1
    print(evaluation_report)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#models = [RandomForestClassifier(), DecisionTreeClassifier(), KNeighborsClassifier()]
models = [DecisionTreeClassifier(), KNeighborsClassifier()]

parameters_list = [
    {#'clf__estimator': [DecisionTreeClassifier()],
     'clf__estimator__criterion': ["gini", "entropy"],
     'clf__estimator__max_depth': [np.arange(5,20)]
    },
    {#'clf__estimator': [KNeighborsClassifier()],
     'clf__estimator__n_neighbors': [np.arange(3,10)]
    }
] 



In [ ]:
def perform_gridsearch_model_and_parameters(model, parameters):
    '''
    Function for gridsearch on model and parameters.
    
    ARGS:
    model:
    parameters: parameters in gridsearch for model
    
    OUTPUT:
    cv: GridSearch model
    
    '''  
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(model))
    ])
    cv = GridSearchCV(pipeline,parameters)
    
    return cv

def evaluate_model_performance(model, X_test, y_test, output_categories):
    y_pred_gridsearch = model.predict(X_test)

    # Todo:  iterating through the category columns
    index = 0
    for category in output_categories:
        print("output category in column {}: {}".format(index, category))
        evaluation_report = classification_report(y_test[:,index], y_pred_gridsearch[:,index])
        index += 1
        print(evaluation_report)
        


In [ ]:
with active_session():
    # do long-running work here
    for model, parameters in zip(models, parameters_list):
        cv = perform_gridsearch_model_and_parameters(model, parameters)
        cv.fit(X_train_debug, y_train_debug)
        print("\nBest Parameters:", cv.best_params_)
        print("Best score: ", cv.best_score_)
        best_model = cv.best_estimator_
        evaluate_model_performance(best_model, X_test, y_test, output_categories)

### 9. Export your model as a pickle file

In [24]:
filename = "classifier.pkl"
pickle.dump(best_model, open(filename, "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
'''
parameters_list = [
    '''
    {'clf__estimator': [RandomForestClassifier()],
     'clf__estimator__n_estimators': [50, 100, 200],
     'clf__estimator__criterion': ["gini", "entropy"],     
     'clf__estimator__bootstrap': [True, False],
     'clf__estimator__max_depth': [10, 50, 100, None],
     'clf__estimator__max_features': ['auto', 'sqrt'],
     'clf__estimator__min_samples_leaf': [1, 2, 4],
     'clf__estimator__min_samples_split': [2, 5, 10],
     'clf__estimator__n_jobs':[-1]
    }
    '''
    {'clf__estimator': [DecisionTreeClassifier()],
     'clf__estimator__criterion': ["gini", "entropy"],
     'clf__estimator__max_depth': [np.arange(5,20)]
    },
    {'clf__estimator': [KNeighborsClassifier()],
     'clf__estimator__n_neighbors': [np.arange(3,10)],
     'clf__estimator__n_jobs':[-1]
    },
]    
'''